In [ ]:
import cv2
import os
import re
import numpy as np

# directories
INPUT_DIR = r"C:\Users\User\Desktop\Mirkwood\images"
OUTPUT_DIR = r"C:\Users\User\Desktop\Mirkwood\images_channels"
MASK_DIR = r"C:\Users\User\Desktop\Mirkwood\mask"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# functions
def extract_number(name):
    match = re.findall(r'\d+', name)
    return match[0] if match else "0"

def apply_mask(channel, mask):
    return cv2.bitwise_and(channel, channel, mask=mask)

def save_channels(img, mask, base_name, color_space, channel_names, channels):
    for i, ch in enumerate(channels):
        ch_masked = apply_mask(ch, mask)
        filename = f"{base_name}_{color_space}_{channel_names[i]}.png"
        path = os.path.join(OUTPUT_DIR, filename)
        cv2.imwrite(path, ch_masked)

# main code
for filename in os.listdir(INPUT_DIR):
    if not filename.lower().endswith((".jpg", ".png", ".tif", ".tiff")):
        continue

    number = extract_number(filename)
    base_name = f"img{number}"
    image_path = os.path.join(INPUT_DIR, filename)

    # load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load: {filename}")
        continue

    # load corresponding mask
    mask_file = next((f for f in os.listdir(MASK_DIR) if number in f), None)
    if mask_file is None:
        print(f"No mask found for: {filename}")
        continue

    mask_path = os.path.join(MASK_DIR, mask_file)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None or mask.shape != img.shape[:2]:
        print(f"Invalid mask for: {filename}")
        continue

    # ensure binary mask (0 or 255)
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    save_channels(gray, mask, base_name, "Gray", ["Gray"], [gray])

    # RGB
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    save_channels(rgb, mask, base_name, "RGB", ["R", "G", "B"], cv2.split(rgb))

    # YUV
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    save_channels(yuv, mask, base_name, "YUV", ["Y", "U", "V"], cv2.split(yuv))

    # HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    save_channels(hsv, mask, base_name, "HSV", ["H", "S", "V"], cv2.split(hsv))

    # HSL (HLS in OpenCV)
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    save_channels(hls, mask, base_name, "HSL", ["H", "L", "S"], cv2.split(hls))

    # CIELab
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    save_channels(lab, mask, base_name, "CIELab", ["L", "A", "B"], cv2.split(lab))

    # YCrCb
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    save_channels(ycrcb, mask, base_name, "YCrCb", ["Y", "Cr", "Cb"], cv2.split(ycrcb))

    print(f"Processed with ROI: {filename}")

print("All images processed with masks.")